In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [21]:
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
import numpy as np
import os
from PIL import Image

def load_hurricane_data(loading_method):
    if loading_method == 'drive':
        data_path = '/content/drive/My Drive/Deep_learning_data/flowers'
    elif loading_method == 'local':
        data_path = 'raw_data/train_another'
    classes = {'no_damage':0, 'damage':1}
    imgs = []
    labels = []
    for (cl, i) in classes.items():
        images_path = [elt for elt in os.listdir(os.path.join(data_path, cl)) if elt.find('.jpeg')>0]
        for img in tqdm(images_path[:300]):
            path = os.path.join(data_path, cl, img)
            if os.path.exists(path):
                image = Image.open(path)
                image = image.resize((128, 128))
                imgs.append(np.array(image))
                labels.append(i)

    X = np.array(imgs)
    #num_classes = len(set(labels))
    #print(X)
    #print(num_classes)
    #print(labels)
    #print(len(labels))
    #y = to_categorical(labels, num_classes)
    y = np.array(labels)

    # Finally we shuffle:
    p = np.random.permutation(len(X))
    X, y = X[p], y[p]

    first_split = int(len(imgs) /6.)
    second_split = first_split + int(len(imgs) * 0.2)
    X_test, X_val, X_train = X[:first_split], X[first_split:second_split], X[second_split:]
    y_test, y_val, y_train = y[:first_split], y[first_split:second_split], y[second_split:]
    
    return X_train, y_train, X_val, y_val, X_test, y_test

In [22]:
X_train, y_train, X_val, y_val, X_test, y_test = load_hurricane_data('local')

100%|████████████████████████████████████████| 300/300 [00:00<00:00, 570.06it/s]


In [27]:
X_train = X_train / 255
X_val = X_val / 255
X_test = X_test / 255

In [28]:
print(X_train.shape, X_val.shape, X_test.shape)

(380, 128, 128, 3) (120, 128, 128, 3) (100, 128, 128, 3)


In [29]:
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import models, layers

In [36]:
def model_init():
    
    model = models.Sequential()

    model.add(Rescaling(1/255, input_shape = (128, 128, 3)))

    model.add(layers.Conv2D(10, kernel_size = 8, activation = 'relu'))
    model.add(layers.MaxPooling2D(4))

    model.add(layers.Flatten())

    model.add(layers.Dense(1, activation = 'sigmoid'))
    
    return model

In [38]:
model = model_init()
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_4 (Rescaling)      (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 121, 121, 10)      1930      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 10)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9001      
Total params: 10,931
Trainable params: 10,931
Non-trainable params: 0
_________________________________________________________________


In [44]:
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
             metrics = ['accuracy'])

In [45]:
model.fit(X_train, 
          y_train,
          batch_size = 32,
          epochs = 10)

Epoch 1/10
12/12 [==============================] - 2s 163ms/step - loss: 0.6933 - accuracy: 0.5105
Epoch 2/10
12/12 [==============================] - 2s 172ms/step - loss: 0.6931 - accuracy: 0.5105
Epoch 3/10
12/12 [==============================] - 2s 156ms/step - loss: 0.6931 - accuracy: 0.5105
Epoch 4/10
12/12 [==============================] - 2s 161ms/step - loss: 0.6931 - accuracy: 0.5105
Epoch 5/10
12/12 [==============================] - 2s 161ms/step - loss: 0.6931 - accuracy: 0.5105
Epoch 6/10
12/12 [==============================] - 2s 164ms/step - loss: 0.6931 - accuracy: 0.5105
Epoch 7/10
12/12 [==============================] - 2s 203ms/step - loss: 0.6931 - accuracy: 0.5105
Epoch 8/10
12/12 [==============================] - 2s 193ms/step - loss: 0.6931 - accuracy: 0.5105
Epoch 9/10
12/12 [==============================] - 3s 241ms/step - loss: 0.6930 - accuracy: 0.5105
Epoch 10/10
12/12 [==============================] - 3s 235ms/step - loss: 0.6930 - accuracy: 0.5105

In [46]:
model.evaluate(X_test, y_test)

4/4 [==============================] - 0s 20ms/step - loss: 0.6932 - accuracy: 0.5200


[0.693192720413208, 0.5199999809265137]